In [ ]:
import os
import json
import time
import requests
from datetime import datetime
from datetime import date
#import dateutil
import pandas as pd
#from dateutil.relativedelta import relativedelta
import numpy as np

# Choose time period of interest
start_year=1900
start_month=1
end_year=2010
end_month=12

binary =2
score =1

# to predict binary (whether market goes up or down use binary, to predict sp 500 score use score)
import datetime
spdata=np.loadtxt(open("sp500binary.csv", "rb"), delimiter=",", skiprows=1,usecols=(score))

# function to give inputs for sp500 database
def train_years(mstart, ystart, mstop, ystop, spdata):
    start=(ystart-1871)*12+(mstart-1)
    stop=(ystop-1871)*12+(mstop-1)
    sp=spdata[start:(stop+1)]
    
    return sp

# function to give inputs for NYT database 
def train_years_NYT(mstart, ystart, mstop, ystop):
    start=datetime.date(ystart, mstart, 1)
    end=datetime.date(ystop, mstop, 1)
    months_in_range = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime("%Y %#m").tolist()]
    
    return months_in_range
    
from numpy import load
# load dict of arrays
dict_data = load('data.npz')
# extract the first array
tweets = dict_data['arr_0']

Ret=(train_years(start_month, start_year, end_month, end_year, spdata))

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding

max_words = 100000
maxlen=100000

model = models.Sequential()
model.add(layers.Embedding(max_words, 100, input_length=maxlen))
model.add(layers.Conv1D(80, 100, activation='relu',padding='same'))
#model.add(layers.LSTM(32, return_sequences=True))
model.add(layers.MaxPooling1D(100))   
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))
          
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

start_year=1900
start_month=1
end_year=2010
end_month=12
Ret1=(train_years(start_month, start_year, 11, 1949, spdata))
Ret2=(train_years(start_month, start_year, end_month, end_year, spdata))
total_months=(np.shape(Ret2)[0]-np.shape(Ret1)[0])
predicted_score=[]
for i in range(total_months):
    train_months=np.shape(Ret1)[0]+i
    X_train=tweets[:train_months]
    X_test=tweets[train_months:(train_months+1)]
    Y_train=Ret2[:train_months]
    Y_test=Ret2[train_months:(train_months+1)]
    history= model.fit(X_train,
                  Y_train,
                  epochs=10,
                  batch_size=total_months,
                  steps_per_epoch = len(X_train)//total_months)
                 # validation_data=(X_test, Y_test))
    predictions = model.predict(X_test)
    predicted_score.append(predictions[0,0])
    print(predicted_score)